In [1]:

import numpy as np
import pandas as pd
from pathlib import Path
import gc


# xsuite
import xtrack as xt
import xmask as xm
import xfields as xf
import xpart as xp


import BBStudies.Tracking.XsuitePlus as xPlus
import BBStudies.Tracking.InteractionPoint as inp
import BBStudies.Physics.Detuning as tune
import BBStudies.Plotting.BBPlots as bbplt
import BBStudies.Physics.Base as phys
import BBStudies.Physics.Constants as cst





# Importing Collider and Twiss
#-------------------------------------
collider = xt.Multiline.from_json('colliders/collider_BUNCH_0000.json')
twiss = {}
twiss['lhcb1'] = collider['lhcb1'].twiss().to_pandas()
twiss['lhcb2'] = collider['lhcb2'].twiss().reverse().to_pandas()
#-------------------------------------


# Filtering twiss to get rid of slices, entries and exits
#-------------------------------------
light_twiss = {}
for sequence in ['lhcb1','lhcb2']:
    light_twiss[sequence] = xPlus.filter_twiss(twiss[sequence].set_index('name'),entries=['drift','..','_entry','_exit']).reset_index()
#-------------------------------------

Done loading line from dict.           
Done loading line from dict.           
Done loading line from dict.           
Done loading line from dict.           
Compiling ContextCpu kernels...


4df2c2953bab4b8f9225a6be04e36725.c:5275: warning: "_GNU_SOURCE" redefined
 5275 | #  define _GNU_SOURCE // enable GNU libc NAN extension if possible
      | 
In file included from 4df2c2953bab4b8f9225a6be04e36725.c:50:
/storage-hpc/gpfs_data/HPC/home_recovery/phbelang/abp/BBStudies/Executables/miniforge3/envs/py-BB/include/python3.10/pyconfig.h:1621: note: this is the location of the previous definition
 1621 | #define _GNU_SOURCE 1
      | 
4df2c2953bab4b8f9225a6be04e36725.c: In function 'compt_do':
4df2c2953bab4b8f9225a6be04e36725.c:18442:12: warning: unused variable 'e_loss_primary_tot' [-Wunused-variable]
18442 |     double e_loss_primary_tot = 0.0;  // [GeV] total energy lost by the macroparticle
      |            ^~~~~~~~~~~~~~~~~~
4df2c2953bab4b8f9225a6be04e36725.c:18439:62: warning: variable 'ps_e_prime' set but not used [-Wunused-but-set-variable]
18439 |     double e_e_prime, px_e_prime, py_e_prime, pzeta_e_prime, ps_e_prime, pt_e_prime;  // [GeV, 1, 1, 1, 1, 1] scattered pr

Done compiling ContextCpu kernels.
Compiling ContextCpu kernels...


87324a4fcb9f497898708c5fd23fd771.c:5275: warning: "_GNU_SOURCE" redefined
 5275 | #  define _GNU_SOURCE // enable GNU libc NAN extension if possible
      | 
In file included from 87324a4fcb9f497898708c5fd23fd771.c:50:
/storage-hpc/gpfs_data/HPC/home_recovery/phbelang/abp/BBStudies/Executables/miniforge3/envs/py-BB/include/python3.10/pyconfig.h:1621: note: this is the location of the previous definition
 1621 | #define _GNU_SOURCE 1
      | 
87324a4fcb9f497898708c5fd23fd771.c: In function 'compt_do':
87324a4fcb9f497898708c5fd23fd771.c:18442:12: warning: unused variable 'e_loss_primary_tot' [-Wunused-variable]
18442 |     double e_loss_primary_tot = 0.0;  // [GeV] total energy lost by the macroparticle
      |            ^~~~~~~~~~~~~~~~~~
87324a4fcb9f497898708c5fd23fd771.c:18439:62: warning: variable 'ps_e_prime' set but not used [-Wunused-but-set-variable]
18439 |     double e_e_prime, px_e_prime, py_e_prime, pzeta_e_prime, ps_e_prime, pt_e_prime;  // [GeV, 1, 1, 1, 1, 1] scattered pr

Done compiling ContextCpu kernels.


In [6]:
collider.metadata.keys()

dict_keys(['config_J000', 'config_J001'])

In [5]:
import rich

rich.inspect(collider.metadata['config_J001'])

╭───────────────────────────────────────── <class 'dict'> ──────────────────────────────────────────╮
│ dict() -> new empty dictionary                                                                    │
│ dict(mapping) -> new dictionary initialized from a mapping object's                               │
│     (key, value) pairs                                                                            │
│ dict(iterable) -> new dictionary initialized as if via:                                           │
│     d = {}                                                                                        │
│     for k, v in iterable:                                                                         │
│         d[k] = v                                                                                  │
│ dict(**kwargs) -> new dictionary initialized with the name=value pairs                            │
│     in the keyword argument list.  For example:  dict(one=1, two=2)                               │
│                                                                                                   │
│ ╭───────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ {                                                                                             │ │
│ │ │   'from_collider': '/home/HPC/phbelang/abp/BBStudies/study_signature/colliders/c'+16,       │ │
│ │ │   'save_collider': '/home/HPC/phbelang/abp/BBStudies/study_signature/colliders/c'+23,       │ │
│ │ │   'log_file': 'tree_maker.log',                                                             │ │
│ │ │   'config_collider': {                                                                      │ │
│ │ │   │   'config_knobs_and_tuning': {                                                          │ │
│ │ │   │   │   'knob_settings': {                                                                │ │
│ │ │   │   │   │   'on_x1': -160.0,                                                              │ │
│ │ │   │   │   │   'on_sep1': 0.0,                                                               │ │
│ │ │   │   │   │   'on_x2h': 0.0,                                                                │ │
│ │ │   │   │   │   'on_sep2h': 0.0,                                                              │ │
│ │ │   │   │   │   'on_x2v': 200.0,                                                              │ │
│ │ │   │   │   │   'on_sep2v': 0.0,                                                              │ │
│ │ │   │   │   │   'on_x5': 160.0,                                                               │ │
│ │ │   │   │   │   'on_sep5': 0.0,                                                               │ │
│ │ │   │   │   │   'on_x8h': -200.0,                                                             │ │
│ │ │   │   │   │   'on_sep8h': 0.0,                                                              │ │
│ │ │   │   │   │   ... +13                                                                       │ │
│ │ │   │   │   },                                                                                │ │
│ │ │   │   │   'closed_orbit_correction': {                                                      │ │
│ │ │   │   │   │   'lhcb1': '_correction/corr_co_lhcb1.json',                                    │ │
│ │ │   │   │   │   'lhcb2': '_correction/corr_co_lhcb2.json'                                     │ │
│ │ │   │   │   },                                                                                │ │
│ │ │   │   │   'qx': {'lhcb1': 62.31, 'lhcb2': 62.31},                                           │ │
│ │ │   │   │   'qy': {'lhcb1': 60.32, 'lhcb2': 60.32},                                           │ │
│ │ │   │   │   'dqx': {'lhcb1': 15.0, 'lhcb2': 15.0},                                            │ │
│ │ │   │   │   'dqy': {'lhcb1': 15.0, 'lhcb2': 15.0},                                            │ │
│ │ │   │   │   'delta